#### Two-Stage Evaluation
For production-ready model, we view its quality issues divisible. Objective quality issues and Subjective quality issues.
* Objective Issues: Harmful & Illegal response, In-coherent response, Out-of-Character response 
These issues can be "Detected" through simulation & detection. Currently Detection script is specific for FWD use, as it involves spinninig up vLLM instance and detect specific issues for the deployed model. 

* Subjective Evaluation: Whether the response is funny enough, whether the response is xxx enough
These issues require aligning with human preference. 

In [1]:
from utils import collect_ooc_response
collect_ooc_response() 

In [1]:
from src.attribute import Requirement, Attribute

requirements = [
    {
        "name": "Funny",
        "desc": "Assistant should be funny.",
    },
    {
        "name": "Concise",
        "desc": "Response from assistant should be concise and within 2 sentences",
    }
]

requirement = Requirement.make(requirements)

In [3]:
# Now we build the evaluator -- Comparative Score (Base on comparing with the buckets)
from src.eval import AOEval 

aoeval = AOEval(requirement)

In [5]:
aoeval.human_annotation("haha", requirement.attributes[0], "Ok")

Attribute: Funny
Conversation:
haha
Suggested bucket: Ok


'Ok'